### Hard Problem (Google) - How many Travelers reaching back Home

In [38]:
import pandas as pd
import sqlite3
import sql
sql.style = 'plain'
%load_ext sql
connection=sqlite3.connect("question_bank.db")
print("Successfully connected to SQL database")
# Connect to SQLite
%sql sqlite:///question_bank.db
print("success")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Successfully connected to SQL database
success


In [39]:
%%sql
CREATE TABLE travel_history (date DATE, start_city VARCHAR(50), end_city VARCHAR(50), traveler VARCHAR(50));

INSERT INTO travel_history (date, start_city, end_city, traveler) VALUES ('2024-01-01', 'Delhi', 'Dubai', 'Amit'), ('2024-01-05', 'Dubai', 'London', 'Amit'), ('2024-01-10', 'London', 'Delhi', 'Amit'), ('2024-02-01', 'Mumbai', 'Singapore', 'Priya'), ('2024-02-05', 'Singapore', 'Sydney', 'Priya'), ('2024-02-10', 'Sydney', 'New York', 'Priya'), ('2024-03-01', 'Kolkata', 'Bangkok', 'Raj'), ('2024-03-03', 'Bangkok', 'Tokyo', 'Raj'), ('2024-03-07', 'Tokyo', 'Kolkata', 'Raj'), ('2024-04-01', 'Bangalore', 'Paris', 'Neha'), ('2024-04-05', 'Paris', 'Rome', 'Neha'), ('2024-04-10', 'Rome', 'Berlin', 'Neha'), ('2024-05-01', 'Chennai', 'Dubai', 'Arjun'), ('2024-05-03', 'Dubai', 'Amsterdam', 'Arjun'), ('2024-05-06', 'Amsterdam', 'Chennai', 'Arjun'); 

 * sqlite:///question_bank.db
(sqlite3.OperationalError) table travel_history already exists
[SQL: CREATE TABLE travel_history (date DATE, start_city VARCHAR(50), end_city VARCHAR(50), traveler VARCHAR(50));]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [40]:
%%sql
SELECT * FROM travel_history

 * sqlite:///question_bank.db
Done.


date,start_city,end_city,traveler
2024-01-01,Delhi,Dubai,Amit
2024-01-05,Dubai,London,Amit
2024-01-10,London,Delhi,Amit
2024-02-01,Mumbai,Singapore,Priya
2024-02-05,Singapore,Sydney,Priya
2024-02-10,Sydney,New York,Priya
2024-03-01,Kolkata,Bangkok,Raj
2024-03-03,Bangkok,Tokyo,Raj
2024-03-07,Tokyo,Kolkata,Raj
2024-04-01,Bangalore,Paris,Neha


* A group of travelers embark on world tours starting with their home cities. 
* Each traveler has an undecided itinerary that evolves over the course of the tour. 
* Some travelers decide to abruptly end their journey mid-travel and live in their last destination.
* Given the dataset of dates on which they travelled between different pairs of cities, 
* can you find out how many travellers ended back in their home city? For simplicity, 
* you can assume that each traveler made at most one trip between two cities in a day.

In [50]:
%%sql
SELECT *,
    ROW_NUMBER() OVER (PARTITION BY traveler ORDER BY date) as rk
FROM travel_history

 * sqlite:///question_bank.db
Done.


date,start_city,end_city,traveler,rk
2024-01-01,Delhi,Dubai,Amit,1
2024-01-05,Dubai,London,Amit,2
2024-01-10,London,Delhi,Amit,3
2024-05-01,Chennai,Dubai,Arjun,1
2024-05-03,Dubai,Amsterdam,Arjun,2
2024-05-06,Amsterdam,Chennai,Arjun,3
2024-04-01,Bangalore,Paris,Neha,1
2024-04-05,Paris,Rome,Neha,2
2024-04-10,Rome,Berlin,Neha,3
2024-02-01,Mumbai,Singapore,Priya,1


In [56]:
%%sql

-- Amit starts from delhi and ends at delhi - he retunrs home
-- Same goes for Raj and Arjun
-- Priya and Neha doesnt returns to thier hometown
-------------------------------------------------------------------------------

-- Here group by doesn;t work - we need to group by without losing on info
-- both three enteries for each traveler is important, so use window function
-- Can use any ranking window function, since there no duplicates or same dates
-- We;ll go for ROW_NUMBER - GIVES UNIQUE RANK TO EACH VALUE
-------------------------------------------------------------------------------

## Query

SELECT 
    traveler,date,start_city,end_city,
    ROW_NUMBER() OVER (PARTITION BY traveler ORDER BY date asc) as rank_dates_top_bottom,
    ROW_NUMBER() OVER (PARTITION BY traveler ORDER BY date desc) as rank_dates_bottom_top
FROM travel_history

## Query

-- For amit, ranking his trip dates in asc order, eventually ranking his start_cities from top to bottom
-- Similaryly, ranking his trip dates in desc order, ranking his end_cities from bottom to top

-- For amit
    -- date in asc 
        --> Delhi  - 1
        --> Dubai  - 2
        --> London - 3

    -- date in desc
        --> Dubai  -  3
        --> London -  2
        --> Delhi  -  1

 * sqlite:///question_bank.db
Done.


traveler,date,start_city,end_city,rank_dates_top_bottom,rank_dates_bottom_top
Amit,2024-01-01,Delhi,Dubai,1,3
Amit,2024-01-05,Dubai,London,2,2
Amit,2024-01-10,London,Delhi,3,1
Arjun,2024-05-01,Chennai,Dubai,1,3
Arjun,2024-05-03,Dubai,Amsterdam,2,2
Arjun,2024-05-06,Amsterdam,Chennai,3,1
Neha,2024-04-01,Bangalore,Paris,1,3
Neha,2024-04-05,Paris,Rome,2,2
Neha,2024-04-10,Rome,Berlin,3,1
Priya,2024-02-01,Mumbai,Singapore,1,3


In [72]:
%%sql
WITH ranked_trips AS(
    SELECT 
        traveler,date,start_city,end_city,
        ROW_NUMBER() OVER (PARTITION BY traveler ORDER BY date asc) as rank_dates_top_bottom,
        ROW_NUMBER() OVER (PARTITION BY traveler ORDER BY date desc) as rank_dates_bottom_top
    FROM travel_history
),
TEMP AS(
    SELECT 
        traveler,
        start_city,end_city,date,
        rank_dates_top_bottom,
        rank_dates_bottom_top,
        CASE WHEN rank_dates_top_bottom=1 THEN start_city end as hometown,
        CASE WHEN rank_dates_bottom_top=1 THEN end_city end as final_town
    FROM ranked_trips
)
SELECT * FROM TEMP

 * sqlite:///question_bank.db
Done.


traveler,start_city,end_city,date,rank_dates_top_bottom,rank_dates_bottom_top,hometown,final_town
Amit,Delhi,Dubai,2024-01-01,1,3,Delhi,None
Amit,Dubai,London,2024-01-05,2,2,None,None
Amit,London,Delhi,2024-01-10,3,1,None,Delhi
Arjun,Chennai,Dubai,2024-05-01,1,3,Chennai,None
Arjun,Dubai,Amsterdam,2024-05-03,2,2,None,None
Arjun,Amsterdam,Chennai,2024-05-06,3,1,None,Chennai
Neha,Bangalore,Paris,2024-04-01,1,3,Bangalore,None
Neha,Paris,Rome,2024-04-05,2,2,None,None
Neha,Rome,Berlin,2024-04-10,3,1,None,Berlin
Priya,Mumbai,Singapore,2024-02-01,1,3,Mumbai,None


In [65]:
%%sql
WITH ranked_trips AS(
    SELECT 
        traveler,date,start_city,end_city,
        ROW_NUMBER() OVER (PARTITION BY traveler ORDER BY date asc) as rank_dates_top_bottom,
        ROW_NUMBER() OVER (PARTITION BY traveler ORDER BY date desc) as rank_dates_bottom_top
    FROM travel_history
),
TEMP AS(
    SELECT 
        traveler,
        start_city,end_city,date,
        rank_dates_top_bottom,
        rank_dates_bottom_top,
        max(CASE WHEN rank_dates_top_bottom=1 THEN start_city end) as hometown,
        max(CASE WHEN rank_dates_bottom_top=1 THEN end_city end) as final_town
    FROM ranked_trips
    GROUP BY traveler                           -- max without group by will give any max value in that column, so group by
)
SELECT * FROM TEMP

 * sqlite:///question_bank.db
Done.


traveler,start_city,end_city,date,rank_dates_top_bottom,rank_dates_bottom_top,hometown,final_town
Amit,London,Delhi,2024-01-10,3,1,Delhi,Delhi
Arjun,Amsterdam,Chennai,2024-05-06,3,1,Chennai,Chennai
Neha,Rome,Berlin,2024-04-10,3,1,Bangalore,Berlin
Priya,Sydney,New York,2024-02-10,3,1,Mumbai,New York
Raj,Tokyo,Kolkata,2024-03-07,3,1,Kolkata,Kolkata


* So, now we have got thier city they started from and thier final destination and 
* weather they have returned home or not, should be extracted from these rows

In [73]:
%%sql
WITH ranked_trips AS(
    SELECT 
        traveler,date,start_city,end_city,
        ROW_NUMBER() OVER (PARTITION BY traveler ORDER BY date asc) as rank_dates_top_bottom,
        ROW_NUMBER() OVER (PARTITION BY traveler ORDER BY date desc) as rank_dates_bottom_top
    FROM travel_history
),
TEMP AS(
    SELECT 
        traveler,
        start_city,end_city,date,
        rank_dates_top_bottom,
        rank_dates_bottom_top,
        max(CASE WHEN rank_dates_top_bottom=1 THEN start_city end) as hometown,
        max(CASE WHEN rank_dates_bottom_top=1 THEN end_city end) as final_town
    FROM ranked_trips
    GROUP BY traveler
)
SELECT * FROM TEMP
WHERE hometown=final_town

 * sqlite:///question_bank.db
Done.


traveler,start_city,end_city,date,rank_dates_top_bottom,rank_dates_bottom_top,hometown,final_town
Amit,London,Delhi,2024-01-10,3,1,Delhi,Delhi
Arjun,Amsterdam,Chennai,2024-05-06,3,1,Chennai,Chennai
Raj,Tokyo,Kolkata,2024-03-07,3,1,Kolkata,Kolkata


In [74]:
%%sql
WITH ranked_trips AS(
    SELECT 
        traveler,date,start_city,end_city,
        ROW_NUMBER() OVER (PARTITION BY traveler ORDER BY date asc) as rank_dates_top_bottom,
        ROW_NUMBER() OVER (PARTITION BY traveler ORDER BY date desc) as rank_dates_bottom_top
    FROM travel_history
),
TEMP AS(
    SELECT 
        traveler,
        start_city,end_city,date,
        rank_dates_top_bottom,
        rank_dates_bottom_top,
        max(CASE WHEN rank_dates_top_bottom=1 THEN start_city end) as hometown,
        max(CASE WHEN rank_dates_bottom_top=1 THEN end_city end) as final_town
    FROM ranked_trips
    GROUP BY traveler
)
SELECT COUNT(*) AS itney_ghar_poch_gaye
FROM TEMP
WHERE hometown=final_town

 * sqlite:///question_bank.db
Done.


itney_ghar_poch_gaye
3


## Major Learnings
1. ROW_NUMBER() usage

ROW_NUMBER() OVER (PARTITION BY … ORDER BY …) is super powerful for finding “first” and “last” values per group.

Ordering ASC vs DESC lets you pick earliest vs latest records.

🔹 2. Using CASE WHEN … inside Aggregates

CASE WHEN condition THEN value END inside an aggregate (like MAX) is a common trick to extract conditional values.

Here, you used it to pick only the row with rank=1.

🔹 3. Why MAX() was needed

Not because you wanted the "maximum city," but because you needed an aggregate function to collapse rows after GROUP BY.

It ignores NULLs → perfect for picking out the one non-null value from the CASE expression.

🔹 4. Role of GROUP BY

Without GROUP BY traveler, you’d either get an error or one combined row for the whole dataset.

GROUP BY gives you one row per traveler, which matches the business problem (“find hometown and final town per traveler”).

🔹 5. Output logic (the 1s, 2s, 3s)

ROW_NUMBER() creates the ascending/descending rankings.

You learned why “1” = first trip in ascending order (hometown) and “1” = last trip in descending order (final town).

The “3s” are just the opposite ends of the window.